In [1]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import spacy
import en_core_web_sm
import string

# reviews = pd.read_csv('Datafiniti_Hotel_Reviews.csv')
reviews = pd.read_csv('covid_500.csv', error_bad_lines=False)
reviews.head(2)


C:\Users\Philip\AppData\Local\Temp\ipykernel_9100\1217966314.py:11: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  reviews = pd.read_csv('covid_500.csv', error_bad_lines=False)
b'Skipping line 12: expected 2 fields, saw 3\nSkipping line 30: expected 2 fields, saw 3\nSkipping line 31: expected 2 fields, saw 3\nSkipping line 40: expected 2 fields, saw 5\nSkipping line 45: expected 2 fields, saw 3\nSkipping line 50: expected 2 fields, saw 3\nSkipping line 57: expected 2 fields, saw 3\nSkipping line 58: expected 2 fields, saw 4\nSkipping line 59: expected 2 fields, saw 4\nSkipping line 60: expected 2 fields, saw 3\nSkipping line 62: expected 2 fields, saw 3\nSkipping line 64: expected 2 fields, saw 3\nSkipping line 65: expected 2 fields, saw 3\nSkipping line 67: expected 2 fields, saw 3\nSkipping line 78: expected 2 fields, saw 3\nSkipping line 82: expected 2 fields, saw 4\nSkipping line 85: expec

,Content
Genuine question for health care professionals working in hospitals dealing with COVID (thanks for your incredible work). Going on two most populous states for starters,NSW has 901 people in hospital &amp; Vic has ...
"If the Republicans hadn't spent the last two years spreading COVID disinformation, selling anti-Fauci merchandise, and banning vaccine mandates, the hospitals wouldn't be full right now.\n\nIt's that simple.",NaN


In [2]:
#comments = reviews['reviews.text']
comments = reviews['Content']

comments.head(2)

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Philip\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
#function to remove non-ascii characters
def _removeNonAscii(s): return "".join(i for i in s if ord(i)<128)
comments = comments.astype('str')
#remove non-ascii characters
comments = comments.map(lambda x: _removeNonAscii(x))
#get stop words of all languages
STOPWORDS_DICT = {lang: set(nltk.corpus.stopwords.words(lang)) for lang in nltk.corpus.stopwords.fileids()}
#function to detect language based on # of stop words for particular language
def get_language(text):
    words = set(nltk.wordpunct_tokenize(text.lower()))
    lang = max(((lang, len(words & stopwords)) for lang, stopwords in STOPWORDS_DICT.items()), key = lambda x: x[1])[0]
    if lang == 'english':
        return True
    else:
        return False
#filter for only english comments
eng_comments=comments[comments.apply(get_language)]
eng_comments.head()

Genuine question for health care professionals working in hospitals dealing with COVID (thanks for your incredible work). Going on two most populous states for starters     NSW has 901 people in hospital &amp; Vic has ...
Democrats never thought Covid was real                                                                                                                                       thats why AOC went to Florida even as CNN cla...
Holy shit                                                                                                                                                                    NY is requiring all City &amp; State universi...
@AdrienneTaren Meanwhile                                                                                                                                                     Gov. Parson in Missouri just announced yester...
I am happy to let everyone know that according to the PCR test I took 15 days ago and just got the results from 

In [4]:
#drop duplicates
eng_comments.drop_duplicates(inplace=True)
#load spacy
# nlp = spacy.load('en')
nlp = spacy.load("en_core_web_sm")
# nlp = en_core_web_sm.load()

#function to clean and lemmatize comments
def clean_comments(text):
    #remove punctuations
    regex = re.compile('[' + re.escape(string.punctuation) + '\\r\\t\\n]')
    nopunct = regex.sub(" ", str(text))
    #use spacy to lemmatize comments
    doc = nlp(nopunct, disable=['parser','ner'])
    lemma = [token.lemma_ for token in doc]
    return lemma
#apply function to clean and lemmatize comments
lemmatized = eng_comments.map(clean_comments)
#make sure to lowercase everything
lemmatized = lemmatized.map(lambda x: [word.lower() for word in x])
lemmatized.head(2)


# lemmatized = lemmatized.apply(lambda x: ' '.join([item for item in x if len(item)>2]))
# lemmatized.head()

Genuine question for health care professionals working in hospitals dealing with COVID (thanks for your incredible work). Going on two most populous states for starters    [ , nsw, have, 901, people, in, hospital,  , a...
Democrats never thought Covid was real                                                                                                                                      [ , that, s, why, aoc, go, to, florida, even, ...
Name: Content, dtype: object

In [5]:
unlist_comments = [item for items in lemmatized for item in items]


In [6]:
bigrams = nltk.collocations.BigramAssocMeasures()
trigrams = nltk.collocations.TrigramAssocMeasures()
bigramFinder = nltk.collocations.BigramCollocationFinder.from_words(unlist_comments)
trigramFinder = nltk.collocations.TrigramCollocationFinder.from_words(unlist_comments)

bigram_freq = bigramFinder.ngram_fd.items()
bigramFreqTable = pd.DataFrame(list(bigram_freq), columns=['bigram','freq']).sort_values(by='freq', ascending=False)
bigramFreqTable.head().reset_index(drop=True)

,bigram,freq
0,"(covid, )",10
1,"( , )",7
2,"( , and)",7
3,"(in, the)",7
4,"( , we)",7


In [7]:
nltk.download('averaged_perceptron_tagger')
bigramFreqTable[:10]
#get english stopwords
en_stopwords = set(stopwords.words('english'))
#function to filter for ADJ/NN bigrams
def rightTypes(ngram):
    if '-pron-' in ngram or '' in ngram or ' 'in ngram or 't' in ngram:
        return False
    for word in ngram:
        if word in en_stopwords:
            return False
    acceptable_types = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    second_type = ('NN', 'NNS', 'NNP', 'NNPS')
    tags = nltk.pos_tag(ngram)
    if tags[0][1] in acceptable_types and tags[1][1] in second_type:
        return True
    else:
        return False
#filter bigrams
filtered_bi = bigramFreqTable[bigramFreqTable.bigram.map(lambda x: rightTypes(x))]
filtered_bi[:10]

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Philip\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,bigram,freq
821,"(covid, test)",4
641,"(public, health)",3
462,"(vaccine, passport)",3
716,"(vaccine, mandate)",3
72,"(covid, vaccine)",3
196,"(boris, johnson)",3
424,"(long, term)",2
562,"(health, care)",2
975,"(covid, death)",2
265,"(health, minister)",2


In [8]:
trigram_freq = trigramFinder.ngram_fd.items()
trigramFreqTable = pd.DataFrame(list(trigram_freq), columns=['trigram','freq']).sort_values(by='freq', ascending=False)
trigramFreqTable.head().reset_index(drop=True)

,trigram,freq
0,"( , amp, )",6
1,"(there, s, no)",3
2,"(of, people, )",3
3,"( , which, be)",3
4,"( , if, i)",3


In [9]:
trigramFreqTable[:10]


,trigram,freq
7,"( , amp, )",6
121,"(there, s, no)",3
461,"(of, people, )",3
772,"( , which, be)",3
1366,"( , if, i)",3
252,"(there, be, no)",3
1549,"(we, will, have)",2
2078,"(without, a, mask)",2
2072,"(if, i, m)",2
1310,"(for, covid, )",2


In [10]:
def rightTypesTri(ngram):
    if '-pron-' in ngram or '' in ngram or ' 'in ngram or '  ' in ngram or 't' in ngram:
        return False
    for word in ngram:
        if word in en_stopwords:
            return False
    first_type = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    third_type = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    tags = nltk.pos_tag(ngram)
    if tags[0][1] in first_type and tags[2][1] in third_type:
        return True
    else:
        return False
filtered_tri = trigramFreqTable[trigramFreqTable.trigram.map(lambda x: rightTypesTri(x))]
filtered_tri[:10]

,trigram,freq
1278,"(covid, 19, test)",2
1413,"(health, care, worker)",1
1473,"(future, government, support)",1
1457,"(long, term, complication)",1
1506,"(public, health, guidance)",1
1152,"(underlie, health, condition)",1
1079,"(national, zero, covid)",1
1284,"(look, like, common)",1
1228,"(public, 000, people)",1
1210,"(daily, reporting, underscore)",1
